In [ ]:
import sys
sys.path.append('../input/timmmaster')
sys.path.append("../input/tez-lib")

In [ ]:
import tez
import albumentations
import pandas as pd
import cv2
import numpy as np
import timm
import torch.nn as nn
from sklearn import metrics
import torch
from tez.callbacks import EarlyStopping
from tqdm import tqdm
import math

In [ ]:
class args:
    batch_size = 16
    image_size = 384

In [ ]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

# Dataset

In [ ]:
class PawpularDataset:
    def __init__(self, image_paths, dense_features, targets, augmentations):
        self.image_paths = image_paths
        self.dense_features = dense_features
        self.targets = targets
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item):
        image = cv2.imread(self.image_paths[item])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
            
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        
        features = self.dense_features[item, :]
        targets = self.targets[item]
        return {
            "image": torch.tensor(image, dtype=torch.float),
            "features": torch.tensor(features, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.float),
        }
    
    
        
        
    

In [ ]:
class PawpularModel(tez.Model):
    def __init__(self,model_name):
        super().__init__()
        self.model=timm.create_model(model_name,pretrained=False,in_chans=3)
        self.model.head=nn.Linear(self.model.head.in_features,128)
        self.dropout=nn.Dropout(0.1)
        self.dense1=nn.Linear(140,64)
        self.dense2=nn.Linear(64,1)
    def forward(self,image,features,targets=None):
        x=self.model(image)
        x=self.dropout(x)
        x=torch.cat([x,features],dim=1)
        x=self.dense1(x)
        x=self.dense2(x)
        return x,0,{}
    

In [ ]:
test_aug = albumentations.Compose(
    [
        albumentations.Resize(args.image_size, args.image_size, p=1),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

In [ ]:
super_final_predictions=[]
for fold_ in range(10):
    #Intialize the model
    model = PawpularModel(model_name="swin_large_patch4_window12_384")
    
    #load model from path
    model.load(f"../input/paw-models/model_f{fold_}.bin", device="cuda", weights_only=True)
    
    #load test.csv file
    df_test=pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
    #create list of path+image ids from the test csv Ids values
    test_img_paths=[f"../input/petfinder-pawpularity-score/test/{id_}.jpg" for id_ in df_test['Id'].values]
    
    #create list of dense features
    dense_features = [
        'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
        'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
    ]
        
    #Create dataset
    test_dataset=PawpularDataset(
        image_paths=test_img_paths,
        dense_features=df_test[dense_features].values,
        targets=np.ones(len(test_img_paths)),
        augmentations=test_aug)
    
    #run data through model
    test_predictions=model.predict(test_dataset,batch_size=2*args.batch_size,n_jobs=-1)
        
        
    
    #put predictions in final_test_predictions list
    final_test_predictions=[]
    for preds in tqdm(test_predictions):
        final_test_predictions.extend(preds.ravel().tolist())
        
    #apply sigmoid and multiply by 100 to final_test_predsictions list 
    final_test_predictions=[sigmoid(x)*100 for x in final_test_predictions]
    super_final_predictions.append(final_test_predictions)
        
        
#add the list to super final predictions list
super_final_predictions = np.mean(np.column_stack(super_final_predictions), axis=1)        
#take average over all the folds
    
        





# Mycode

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import sys
import pytorch_lightning as pl
sys.path.append('../input/timmmaster')
import os
import json
import glob
import random
import collections
import albumentations as A
import timm

import numpy as np
import pandas as pd
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import torch
package_path = "../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master/"
import sys 
sys.path.append(package_path)
import glob
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import StratifiedKFold

import time
import os
import torch
from torch import nn
from torch.utils import data as torch_data
from sklearn import model_selection as sk_model_selection
from torch.nn import functional as torch_functional
import torch.optim.lr_scheduler as lr_scheduler

from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm

In [ ]:
def set_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True


set_seed(42)

In [ ]:
df = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
dense_features = [
        'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
        'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
    ]


In [ ]:
import albumentations
transform = albumentations.Compose(
    [
       
        albumentations.HueSaturationValue(
            hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5
        ),
        albumentations.RandomBrightnessContrast(
            brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5
        ),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

valid_aug = albumentations.Compose(
    [
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],)

In [ ]:
class DataRetriever(torch_data.Dataset):
    def __init__(self,ids,targets,train=False,valid=False):
        self.id=ids
        self.targets=targets
        self.train=train
        self.valid=valid
        self.alpha=0.2
        
    def __len__(self):
        return len(self.id)
    def __getitem__(self,index):
        if self.train or self.valid:
            img_path='../input/petfinder-pawpularity-score/train/'+str(self.id[index])+'.jpg'
        else:
            img_path='../input/petfinder-pawpularity-score/test/'+str(self.id[index])+'.jpg'
            
        if self.targets != None:
            labels=torch.tensor(self.targets[index],dtype=torch.float)
        img=cv2.imread(img_path)
        
        img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img=cv2.resize(img,(224,224))
        
        if self.train:
            img=transform(image=img)['image']
            if index%2!=0:
                lam=np.random.beta(self.alpha,self.alpha)
                mixup_index=np.random.randint(0,index)
                labels2=torch.tensor(self.targets[mixup_index],dtype=torch.float)
                img_path='../input/petfinder-pawpularity-score/train/'+str(self.id[mixup_index])+'.jpg'
                img2=cv2.imread(img_path)
                img2=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img2=cv2.resize(img,(224,224))
                img=lam*img+(1-lam)*img2
                labels=lam*labels+(1-lam)*labels2
        else:
            img=valid_aug(image=img)['image']
      
        img=np.transpose(img,(2,0,1)).astype(np.float32)
        if self.train or self.valid:
            return {
            'X':torch.tensor(img,dtype=torch.float),   
            'y':labels
          }
        return{'X':torch.tensor(img,dtype=torch.float)} 

In [ ]:
class PawpularityModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = timm.create_model('swin_large_patch4_window7_224_in22k', pretrained=False, in_chans=3)
        #self.model.classifier = nn.Linear(self.model.classifier.in_features, 128)
        self.dropout = nn.Dropout(0.1)
        self.dense1 = nn.Linear(128, 1)
        #self.dense2 = nn.Linear(64, 1)
        self.model.head=nn.Linear(self.model.head.in_features,128)
        #self.model.fc = nn.Linear(self.model.fc.in_features, 128)
        
    def forward(self,x):
        output=self.model(x)
        #print(output.shape)
        output=self.dropout(output)
        #print(output.shape)
        #output=torch.cat([output,dense_features],dim=1)
        output=self.dense1(output)
        #print(output.shape)
        #output=self.dense2(output)
       # output=torch.sigmoid(output)       
        return output
    


In [ ]:
submission = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")
model=PawpularityModel()
test=pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
model.to('cuda')
device='cuda'
test_data_retriever = DataRetriever(
    submission.Id.values,None,train=False,
    valid=False
)

test_loader = torch_data.DataLoader(
    test_data_retriever,
    batch_size=64,
    shuffle=False,
    num_workers=8,
)

In [ ]:
ids = []
paths=[]
pred=[]
for path in os.listdir('../input/pet-paw-data'):
    paths.append('../input/pet-paw-data/'+path)

for path in paths:
    y_pred = []
    model.load_state_dict(torch.load(path))
    for e, batch in enumerate(test_loader):
        print(f"{e}/{len(test_loader)}", end="\r")
        with torch.no_grad():
            tmp_pred = np.zeros((batch["X"].shape[0], ))

            tmp_pred=model(batch['X'].to(device)).cpu().numpy().squeeze()
            #print(tmp_pred)
            tmp_pred=1/(1+np.exp(-tmp_pred))
           # print(tmp_pred)
            
            y_pred.extend(tmp_pred)
    y_pred=[x*100 for x in y_pred]
    pred.append(y_pred)
    
pred=np.mean(np.column_stack(pred), axis=1)    

In [ ]:
df_test['Pawpularity']=(pred+super_final_predictions)/2
df_test=df_test[['Id','Pawpularity']]
df_test
df_test.to_csv("submission.csv", index=False)   
df_test